In [1]:
import requests
import json
import pandas as pd
import time
from config import API_key
import logging

In [2]:
logging.basicConfig(level=logging.INFO, filename="API_google_log.log",filemode="w",
                    format="%(asctime)s %(levelname)s %(message)s")

In [ ]:
with open('scraping_log.log', 'w'):
    pass # Clearing logs

In [3]:
'''
with open('scraping_log.log', 'w'):
    pass # Clearing logs
'''

"\nwith open('scraping_log.log', 'w'):\n    pass # Clearing logs\n"

In [4]:
all_books = pd.read_csv("All_added_books_data.csv").drop_dublicates()

In [ ]:
titles = all_books['Title'].dropna().tolist()

base_url = "https://www.googleapis.com/books/v1/volumes"
results = []

for title in titles:
    logging.info(f"Started getting info about '{title}' title")
    search_query = title.replace(' ', '+')
    url = f"{base_url}?q=intitle:{search_query}"
    flag = True
    while flag:
      try:
          response = requests.get(url, headers={'accept': 'application/json', 'X-API-KEY': API_key})
          if response.status_code == 200:
              flag = False
              response_data = response.json()

              if 'items' in response_data and len(response_data['items']) > 0:
                  first_result = response_data['items'][0]['volumeInfo']
                  results.append({
                      "Title": title,
                      "API Title": first_result.get('title'),
                      "Authors": first_result.get('authors', []),
                      "Publisher": first_result.get('publisher'),
                      "Published Date": first_result.get('publishedDate'),
                      "Description": first_result.get('description'),
                      "Page Count": first_result.get('pageCount'),
                      "Print Type": first_result.get('printType'),
                      "Categories": first_result.get('categories', []),
                      "Average Rating": first_result.get('averageRating'),
                      "Ratings Count": first_result.get('ratingsCount'),
                      "Language": first_result.get('language'),
                  })
              else:
                  results.append({
                      "Title": title,
                      "API Title": None,
                      "Authors": None,
                      "Publisher": None,
                      "Published Date": None,
                      "Description": None,
                      "Page Count": None,
                      "Print Type": None,
                      "Categories": None,
                      "Average Rating": None,
                      "Ratings Count": None,
                      "Language": None,
                  })
          else:
              logging.warning(f"Error appeared while requesting data for '{title}' title: {response.status_code}")
              time.sleep(1)
      except Exception as e:
          logging.error(f"Error appeared while requesting data for '{title}'", exc_info=e)
          break


    time.sleep(0.2)

In [11]:
results_df = pd.DataFrame(results)
results_df.to_csv("all_api_books.csv", index=False)

In [7]:
results_df

,Title,API Title,Authors,Publisher,Published Date,Description,Page Count,Print Type,Categories,Average Rating,Ratings Count,Language
0,Toofer & The Giblet Season of Light,Toofer,[Tom Blair],iUniverse,2019-09-03,Kreisler’s Credenza Switch-Hitters Such a deal...,213.0,BOOK,[Fiction],NaN,NaN,en
1,Cozy Critters on the Ark (Cozy Critters Storyt...,None,None,None,None,None,NaN,None,None,NaN,NaN,None
2,The Tale of the Other Glove (On Being),"Modesto Mignon, The lily of the valley, and ot...",[Honoré de Balzac],None,1899,None,854.0,BOOK,[],NaN,NaN,en
3,I Love You to the Moon and Back,I Love You to the Moon and Back,[Amelia Hepworth],Tiger Tales,2017-09-05,Show your child just how strong your love is e...,26.0,BOOK,[Juvenile Fiction],NaN,NaN,en
4,The Rainbow Fish,The Rainbow Fish,[Marcus Pfister],Simon and Schuster,1992,Summary: The most beautiful fish in the entire...,32.0,BOOK,[Juvenile Fiction],3.0,5.0,en
...,...,...,...,...,...,...,...,...,...,...,...,...
2528,The Berenstain Bears and Baby Makes Five,The Berenstain Bears and Baby Makes Five,[Stan Berenstain],Random House Books for Young Readers,2011-03-30,Come for a visit in Bear Country with this cla...,34.0,BOOK,[Juvenile Fiction],NaN,NaN,en
2529,Mockingbird Summer: A Novel,Mockingbird Summer,[Lynda Rutledge],Lake Union Publishing,2024-01-30,A powerful and emotional coming-of-age novel s...,0.0,BOOK,[Fiction],NaN,NaN,en
2530,Horton Hears a Who!,Horton Hears a Who!,[Dr. Seuss],RH Childrens Books,2013-09-24,Choose kindness with Horton the elephant and t...,37.0,BOOK,[Juvenile Fiction],5.0,2.0,en
2531,The Night Circus: A Novel,The Night Circus,[Erin Morgenstern],Random House,2011-09-15,THE TIKTOK SENSATION Discover the million-copy...,404.0,BOOK,[Fiction],4.5,16.0,en
